# A short introduction to Topic Modelling using LDAs

In this notebook we will introduce the practical side of LDAs. We have already done most of the work in the previous notebooks, so defining and training LDAs will be a walk in the park, especially when using gensim.

We will process the `shorter_abcnews_text.csv` file in the Drive folder.

## Updating, installing and importing some packages

We will update gensim, and install a visualization tool called `pyldavis`

In [1]:
!pip install --upgrade gensim

     |████████████████████████████████| 23.9 MB 93 kB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [2]:
!pip install pyldavis

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7 MB 75 kB/s 
     |████████████████████████████████| 10.8 MB 52.9 MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=27cc6af15346e45dcbe20ea2569497dd8aae12b98ec596f4b94e9966c42490da
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyldavis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into

In [1]:
from typing import List

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# sns.set_theme()

import nltk
nltk.download("wordnet")
nltk.download("stopwords")
from nltk import WordNetLemmatizer

from wordcloud import WordCloud

from gensim.utils import simple_preprocess
from gensim.models import LdaModel, LdaMulticore
from gensim.corpora import Dictionary

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecate

## Loading and Preprocessing the data

In [2]:
def clean_text(tweet_text: str) -> str:
  """
  Grabs the text of a tweet and removes
  everything that starts with an ampersand.
  """
  words = tweet_text.split()

  clean_words = [word for word in words if not ("@" in word)]
  return " ".join(clean_words)

In [3]:
def preprocess_headlines(col: pd.Series, remove_stopwords: bool = True) -> List[List[str]]:
  """
  Grabs the 'text' column in df and
  returns a preprocessed list of documents.

  See more in previous class' notebooks.
  """
  # Doing a simple preprocess using gensim
  preprocessed_docs = [
    simple_preprocess(t, min_len=0, max_len=100) for t in col
  ]

  # Lemmatizing
  lemmatizer = WordNetLemmatizer()
  preprocessed_docs = [
    [lemmatizer.lemmatize(token) for token in doc] for doc in preprocessed_docs
  ]

  # Removing stopwords
  if remove_stopwords:
    english_stopwords = set(nltk.corpus.stopwords.words("english"))
    preprocessed_docs = [
      [word for word in doc if word not in english_stopwords] for doc in preprocessed_docs
    ]

  return preprocessed_docs


In [4]:
news = pd.read_csv("shorter_abcnews_text.csv")

In [5]:
news.head()

,Unnamed: 0,publish_date,headline_text
0,0,20030219,aba decides against community broadcasting lic...
1,1,20030219,act fire witnesses must be aware of defamation
2,2,20030219,a g calls for infrastructure protection summit
3,3,20030219,air nz staff in aust strike for pay rise
4,4,20030219,air nz strike to affect australian travellers


In [6]:
corpus = preprocess_headlines(news["headline_text"])

In [7]:
corpus[:5]

[['aba', 'decides', 'community', 'broadcasting', 'licence'],
 ['act', 'fire', 'witness', 'must', 'aware', 'defamation'],
 ['g', 'call', 'infrastructure', 'protection', 'summit'],
 ['air', 'nz', 'staff', 'aust', 'strike', 'pay', 'rise'],
 ['air', 'nz', 'strike', 'affect', 'australian', 'traveller']]

## Training an LDA

Because of how it is implemented in `gensim`, we need to define a Dictionary first, get a list with all bag-of-words (using the `Dictionary.doc2bow` method) and training the LDA on this list of all bag-of-words.

In [8]:
dct = Dictionary(corpus)

In [13]:
# There are actually over 1 million headlines, so let's restrict to 10000.
bows = [dct.doc2bow(doc) for doc in corpus[:4000]]

In [14]:
# should take 2.5 minutes.
lda_model = LdaMulticore(
    bows,
    id2word=dct,
    num_topics=10,
    random_state=100,
    chunksize=100,
    passes=10,
    per_word_topics=True
)

## Visualizing and understanding the LDA

We can use pyLDAvis to check the topics in an interactive way.

In [15]:
pyLDAvis.enable_notebook()
lda_viz = gensimvis.prepare(lda_model, bows, dct)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [16]:
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.141864 -0.272533       1        1  12.233957
9     -0.305054 -0.099462       2        1  12.087075
4      0.044376  0.078914       3        1  11.251137
8      0.001476  0.014353       4        1  10.486215
6      0.031894  0.024739       5        1  10.299012
3      0.017709  0.094130       6        1   9.603818
0      0.010077  0.067662       7        1   9.434175
1      0.025841  0.034917       8        1   9.342620
5      0.022491  0.017009       9        1   8.311949
7      0.009326  0.040272      10        1   6.950042, topic_info=         Term        Freq       Total Category  logprob  loglift
45       iraq  137.000000  137.000000  Default  30.0000  30.0000
476       say  129.000000  129.000000  Default  29.0000  29.0000
280    police  115.000000  115.000000  Default  28.0000  28.0000
709         u  119.000000  119.000000  Default  27.0000  27.0000
446       new   71.000000   71.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
1707     duck    7.260503    7.992151  Topic10  -5.3042   2.5704
213      wale    7.187546    7.919061  Topic10  -5.3143   2.5695
2541  officer    7.162200    7.893609  Topic10  -5.3178   2.5692
3313     butt    6.894238    7.625690  Topic10  -5.3559   2.5656
481      stay   10.690540   20.603891  Topic10  -4.9173   2.0103

[343 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1094      3  0.964524  accident
253       5  0.956940   accused
5         7  0.979939       act
618       1  0.932805    action
3238      4  0.947922   advance
...     ...       ...       ...
496       2  0.281832     world
496       4  0.704581     world
1362      1  0.975948      year
449       7  0.918294  zimbabwe
1131     10  0.886228      zone

[318 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 10, 5, 9, 7, 4, 1, 2, 6, 8])

The `lda_model` object itself holds a `.print_topics` method that gives a quick summary of what each `topic` is, related to the tokens.

In [21]:
lda_model.print_topics()

[(0,
  '0.031*"win" + 0.018*"howard" + 0.018*"probe" + 0.016*"vic" + 0.015*"day" + 0.015*"top" + 0.013*"play" + 0.012*"continues" + 0.011*"welcome" + 0.011*"clash"'),
 (1,
  '0.026*"war" + 0.020*"indian" + 0.017*"get" + 0.016*"water" + 0.014*"australia" + 0.013*"green" + 0.013*"nz" + 0.013*"support" + 0.012*"rally" + 0.011*"well"'),
 (2,
  '0.050*"say" + 0.027*"govt" + 0.022*"call" + 0.019*"council" + 0.018*"plan" + 0.014*"year" + 0.012*"fund" + 0.012*"still" + 0.012*"job" + 0.011*"union"'),
 (3,
  '0.024*"man" + 0.020*"anti" + 0.016*"car" + 0.014*"child" + 0.013*"melbourne" + 0.013*"home" + 0.013*"attack" + 0.012*"protest" + 0.012*"bombing" + 0.012*"crash"'),
 (4,
  '0.021*"pm" + 0.019*"wa" + 0.019*"cyclone" + 0.017*"set" + 0.016*"qld" + 0.012*"man" + 0.011*"warning" + 0.011*"north" + 0.011*"back" + 0.011*"accident"'),
 (5,
  '0.021*"hospital" + 0.019*"coast" + 0.018*"nsw" + 0.015*"coalition" + 0.014*"minister" + 0.013*"health" + 0.013*"iraqi" + 0.013*"deal" + 0.013*"public" + 0.012*"

In [24]:
idx = 10


bow = bows[idx]
print(lda_model.get_document_topics(bow))
print(corpus[idx])

[(0, 0.01669908), (1, 0.3488181), (2, 0.01669908), (3, 0.01669908), (4, 0.3505845), (5, 0.01669908), (6, 0.01669908), (7, 0.01669908), (8, 0.01669908), (9, 0.18370381)]
['australia', 'contribute', 'million', 'aid', 'iraq']


In [23]:
corpus[1]

['act', 'fire', 'witness', 'must', 'aware', 'defamation']

In [29]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [28]:
perplexities = []
for k in [5, 10, 15, 20]:
  lda_model = LdaMulticore(
    bows,
    id2word=dct,
    num_topics=k,
    random_state=100,
    chunksize=100,
    passes=10,
    per_word_topics=True
  )
  perplexities.append(lda_model.log_perplexity(bows))

print(perplexities)

[-9.178746672571533, -10.111345102333573, -16.078761754964447, -19.837467083793882]


In [30]:
lda_model = LdaMulticore(
    bows,
    id2word=dct,
    num_topics=100,
    random_state=100,
    chunksize=100,
    passes=10,
    per_word_topics=True
  )

In [31]:
lda_model.log_perplexity(bows)

-177.69207720075332